# Lab4 (Students version): robustness of a graph 

We can use the following libraries.

In [49]:
import matplotlib.pyplot as plt
import math
import sys
import time
import random
import copy
from collections import deque
print(sys.version)

3.9.6 (default, Sep 19 2022, 09:09:38) 
[GCC 10.2.1 20210110]


In this lab session, we investigate the notion of robustness of a graph: how well a graph remains connected when nodes disappear following random failures or degree-based failures.

## Exercise 1: preliminary work

### Question 1

Using the code seen in previous labs, load the following graph as a dictonary of lists:

http://lioneltabourier.fr/documents/inet.txt

In [5]:
# Code taken from our TP1

def load_graph(filename):
    # Initialize an empty dictionary for the adjacency list.
    adjacency_list = {}
    
    # Open the specified file for reading.
    with open(filename, 'r') as file:
        # Loop through each line in the file.
        for line in file:
            # Check if the line does not start with '#' (comments)
            if not line.startswith('#'):
                # Split the line into two nodes representing an edge.
                edge_info = line.strip().split()
                node1, node2 = edge_info[0], edge_info[1]
                # Check if node1 is not already in the adjacency list.
                if node1 not in adjacency_list:
                    # Initialize an empty list for node1 in the adjacency list.
                    adjacency_list[node1] = []
                # Check if node2 is not already in the adjacency list.
                if node2 not in adjacency_list:
                    # Initialize an empty list for node2 in the adjacency list.
                    adjacency_list[node2] = []
                # Add node2 to the adjacency list of node1 (representing an undirected edge).
                adjacency_list[node1].append(node2)
                # Add node1 to the adjacency list of node2 (since it's an undirected edge).
                adjacency_list[node2].append(node1)

    # Return the adjacency list representing the undirected graph.
    return adjacency_list

In [6]:
# Loading our inet.txt into a variable for easier use down the line
filename = 'inet.txt'
start_time = time.time()
inet_graph = load_graph(filename)
print(f"Graph loaded in {time.time() - start_time:.5f} seconds")

Graph loaded in 0.07878 seconds


### Question 2

Determine the size of the largest connected component (LCC) of a graph, and use the code to determine the size of the LCC of the example graph.

Suggested implementation:

- Create a function that takes a graph as input and outputs a dictionary of the connected component that each node belongs to. (This function is derived from a BFS).

- Then, create another function which takes the dictionary of the connected components as input and computes the size of the largest connected component of the graph.

In [7]:
# Code taken from our TP2

def bfs(graph, start_node):
    # Initialize a set to keep track of visited nodes
    visited = set()
    # Initialize a deque (double-ended queue) with the starting node
    queue = deque([start_node])
    # Initialize a list to store the connected component (nodes visited during BFS)
    component = []

    # Main BFS loop
    while queue:
        # Get the current node from the front of the queue
        current = queue.popleft()
        # Check if the current node has not been visited
        if current not in visited:
            # Mark the current node as visited
            visited.add(current)
            # Add the current node to the connected component
            component.append(current)
            # Extend the queue with unvisited neighbors of the current node
            queue.extend(node for node in graph[current] if node not in visited)

    # Return the connected component
    return component

In [8]:
# Code taken from our TP2

def connected_components(graph):
    # Initialize a set to keep track of visited nodes
    visited = set()
    # Initialize a list to store connected components
    components = []

    # Iterate through all vertices in the graph
    for node in graph:
        # Check if the node has not been visited
        if node not in visited:
            # Use BFS to find the connected component starting from the current node
            component = bfs(graph, node)
            # Append the connected component to the list of components
            components.append(component)
            # Mark all nodes in the component as visited
            visited.update(component)

    # Return the list of connected components
    return components

In [9]:
# Code taken from our TP2

def process_graph(graph):
    start_time = time.time()
    # Find the largest connected component
    components = connected_components(graph)
    largest_component = max(components, key=len)
    
    # Display the largest connected component
    if len(largest_component) > 10:
        print(f"Largest connected component: {largest_component[:10]}...{largest_component[-10:]}")
    else:
        print(f"Largest connected component: {largest_component}")
    
    print(f"\nSize of the largest connected component: {len(largest_component)}")
    print("\nAll connected components:")
    
    # Display all connected components
    for i, component in enumerate(components, 1):
        if len(component) > 10:
            print(f"Component {i}: {component[:10]}...{component[-10:]}")
        else:
            print(f"Component {i}: {component}")
    
    print(f"\nConnected components found in {time.time() - start_time:.5f} seconds")

In [10]:
graph = inet_graph
result = process_graph(graph)

Largest connected component: ['0', '1882', '7203', '5853', '6239', '1888', '7163', '7167', '7204', '7206']...['1715', '1717', '1718', '1719', '6903', '3950', '3948', '1714', '1716', '1713']

Size of the largest connected component: 8557

All connected components:
Component 1: ['0', '1882', '7203', '5853', '6239', '1888', '7163', '7167', '7204', '7206']...['1715', '1717', '1718', '1719', '6903', '3950', '3948', '1714', '1716', '1713']
Component 2: ['2', '7880', '7881', '7882', '7878', '7879']
Component 3: ['19', '25', '3178', '3180', '3242', '28', '29', '27', '26']
Component 4: ['20', '3253', '23', '22', '3235', '3177', '3179', '3256', '3250', '3236']...['3266', '3267', '3268', '3269', '3270', '3271', '3272', '3273', '3274', '3276']
Component 5: ['24', '3252', '3246']
Component 6: ['160', '161']
Component 7: ['171', '172', '173', '4743', '174', '175', '176', '177', '178', '179']...['172', '173', '4743', '174', '175', '176', '177', '178', '179', '180']
Component 8: ['184', '185', '4429']

## Exercise 2: robustness to random failures

### Question 3

In this question, we plot the size of the LCC as a function of the number of nodes removed. 

Nodes are removed randomly. It is a way to evaluate the robustness of the network to random failures.

Suggested implementation:

- create a function that deletes $n_s$ nodes from the original graph

- use the function of Question 2 to compute the size of the LCC

- combine these two functions and iterate to get a dictionary which keys are $n_s$ and values are the corresponding size of the LCC




In [38]:
def remove_nodes(graph, n):
    # Make a copy of the original graph
    modified_graph = graph.copy()
    
    # Get a list of nodes to remove randomly
    nodes_to_remove = random.sample(list(graph.keys()), n)
    
    # Remove the selected nodes and their edges
    for node in nodes_to_remove:
        del modified_graph[node]
        for neighbor in modified_graph:
            modified_graph[neighbor] = [x for x in modified_graph[neighbor] if x != node]
    
    return modified_graph

In [39]:
def compute_lcc_size(graph):
    components = connected_components(graph)
    # Find the size of the largest connected component (LCC)
    lcc_size = max(len(component) for component in components)
    return lcc_size

In [40]:
def evaluate_robustness(graph, num_iterations, max_nodes_to_remove):
    robustness_dict = {}
    
    for n in range(1, max_nodes_to_remove + 1):
        lcc_sizes = []
        
        for _ in range(num_iterations):
            modified_graph = remove_nodes(graph, n)
            lcc_size = compute_lcc_size(modified_graph)
            lcc_sizes.append(lcc_size)
        
        # Calculate the average LCC size for n nodes removed
        average_lcc_size = sum(lcc_sizes) / num_iterations
        
        robustness_dict[n] = average_lcc_size
    
    return robustness_dict

In [43]:
graph = inet_graph
num_iterations = 50  # Number of iterations to compute average LCC size
max_nodes_to_remove = 15  # Maximum number of nodes to remove
start_time = time.time()
result = evaluate_robustness(graph, num_iterations, max_nodes_to_remove)
print(result)
print(f"Graph loaded in {time.time() - start_time:.5f} seconds")

{1: 8529.36, 2: 8552.28, 3: 8526.36, 4: 8549.46, 5: 8548.46, 6: 8548.94, 7: 8543.14, 8: 8529.12, 9: 8520.3, 10: 8537.8, 11: 8536.96, 12: 8539.36, 13: 8529.32, 14: 8526.78, 15: 8532.62}
Graph loaded in 33.87378 seconds


## Exercise 3: robustness to targeted (degree-based) failures 

### Question 4

In this question, we do the same as in the previous question, except for the fact that nodes are not chosen randomly, but by decreasing degree order.

Suggested implementation:

- create a function that outputs a list of nodes ordered by decreasing degree

- then follow the same principle as in the previous question

In [44]:
def nodes_by_decreasing_degree(graph):
    # Sort nodes by their degree in decreasing order
    nodes_sorted = sorted(graph.keys(), key=lambda node: -len(graph[node]))
    return nodes_sorted

In [45]:
def evaluate_robustness_decreasing_degree(graph, num_iterations, max_nodes_to_remove):
    robustness_dict = {}
    
    ordered_nodes = nodes_by_decreasing_degree(graph)
    
    for n in range(1, max_nodes_to_remove + 1):
        average_lcc_size = 0
        
        for _ in range(num_iterations):
            modified_graph = remove_nodes_decreasing_degree(graph, ordered_nodes, n)
            components = connected_components(modified_graph)
            lcc_size = max(len(component) for component in components)
            
            # Update the running average directly
            average_lcc_size += (lcc_size - average_lcc_size) / (_ + 1)
        
        robustness_dict[n] = average_lcc_size
    
    return robustness_dict

In [46]:
def remove_nodes_decreasing_degree(graph, ordered_nodes, n):
    modified_graph = {node: neighbors[:] for node, neighbors in graph.items()}
    
    nodes_to_remove = ordered_nodes[:n]
    
    for node in nodes_to_remove:
        del modified_graph[node]
        for neighbor in modified_graph:
            modified_graph[neighbor] = [x for x in modified_graph[neighbor] if x != node]
    
    return modified_graph

In [47]:
graph = inet_graph
num_iterations = 50  # Number of iterations to compute average LCC size
max_nodes_to_remove = 15  # Maximum number of nodes to remove
start_time = time.time()
result = evaluate_robustness_decreasing_degree(graph, num_iterations, max_nodes_to_remove)
print(result)
print(f"Graph loaded in {time.time() - start_time:.5f} seconds")

{1: 8556.0, 2: 8497.0, 3: 8497.0, 4: 8497.0, 5: 8497.0, 6: 8497.0, 7: 8497.0, 8: 8497.0, 9: 8497.0, 10: 8497.0, 11: 8497.0, 12: 8497.0, 13: 8497.0, 14: 8497.0, 15: 8497.0}
Graph loaded in 34.50913 seconds


### Question 5

Compare the two curves (random deletions and targeted deletions): are they different? What does it mean?

In [ ]:
graph = inet_graph
num_iterations = 50  # Number of iterations
max_nodes_to_remove = 10  # Maximum number of nodes to remove

# Evaluate robustness for random deletions
random_robustness = evaluate_robustness(graph, num_iterations, max_nodes_to_remove)

# Evaluate robustness for targeted deletions (decreasing degree order)
targeted_robustness = evaluate_robustness_decreasing_degree(graph, num_iterations, max_nodes_to_remove)

# Extract x and y values for plotting
x_values = list(random_robustness.keys())
random_y_values = list(random_robustness.values())
targeted_y_values = list(targeted_robustness.values())

# Plot the curves
plt.figure(figsize=(10, 6))
plt.plot(x_values, random_y_values, label="Random Deletions")
plt.plot(x_values, targeted_y_values, label="Targeted Deletions (Decreasing Degree Order)")
plt.yscale('log')
plt.xscale('log')
plt.xlabel("Number of Nodes Removed")
plt.ylabel("Size of Largest Connected Component")
plt.title("Network Robustness Comparison")
plt.legend()
plt.grid(True)
plt.show()

### Question 6

Do the same thing but now with targeted (closeness and betweenness based) failures


In [61]:
def closeness_centrality(graph, node):
    shortest_paths = {}
    visited = set()
    queue = [(node, 0)]
    
    while queue:
        current_node, distance = queue.pop(0)
        if current_node not in visited:
            visited.add(current_node)
            if current_node != node:
                shortest_paths[current_node] = distance
            queue.extend((neighbor, distance + 1) for neighbor in graph[current_node] if neighbor not in visited)
    
    total_path_lengths = sum(shortest_paths.values())
    return 1 / total_path_lengths if total_path_lengths > 0 else 0

In [62]:
def betweenness_centrality(graph, node):
    num_shortest_paths = {node: 1}
    dependency = {node: 0}
    stack = []
    
    for start_node in graph:
        if start_node != node:
            num_shortest_paths[start_node] = 0
            dependency[start_node] = 0
    
    queue = list(graph[node])
    
    while queue:
        current_node = queue.pop(0)
        stack.append(current_node)
        
        for neighbor in graph[current_node]:
            if neighbor not in stack and neighbor != node:
                queue.append(neighbor)
            
            if neighbor != node:
                contribution = (num_shortest_paths[current_node] / num_shortest_paths[neighbor]) * (1 + dependency[current_node])
                dependency[neighbor] += contribution
    
    return dependency[node]

In [63]:
def perform_targeted_failures(nodes_sorted, centrality_robustness):
    modified_graph = copy.deepcopy(graph)
    for i in range(1, len(graph)):
        removed_node = nodes_sorted[i - 1]
        del modified_graph[removed_node]
        for node in modified_graph:
            modified_graph[node] = [neighbor for neighbor in modified_graph[node] if neighbor != removed_node]
        centrality_robustness[i] = largest_connected_component_size(modified_graph)

In [64]:
def largest_connected_component_size(graph):
    visited = set()
    sizes = []
    for node in graph:
        if node not in visited:
            component = []
            queue = [node]
            while queue:
                current_node = queue.pop(0)
                if current_node not in visited:
                    visited.add(current_node)
                    component.append(current_node)
                    queue.extend(neighbor for neighbor in graph[current_node] if neighbor not in visited)
            sizes.append(len(component))
    return max(sizes) if sizes else 0

In [65]:
graph = inet_graph

# Sort nodes based on centrality measures in descending order
nodes_sorted_closeness = sorted(graph.keys(), key=lambda node: -closeness_centrality(graph, node))
nodes_sorted_betweenness = sorted(graph.keys(), key=lambda node: -betweenness_centrality(graph, node))

# Initialize dictionaries to store LCC sizes
closeness_robustness = {}
betweenness_robustness = {}

perform_targeted_failures(nodes_sorted_closeness, closeness_robustness)
perform_targeted_failures(nodes_sorted_betweenness, betweenness_robustness)

ZeroDivisionError: division by zero

In [74]:
plt.figure(figsize=(10, 6))
plt.plot(closeness_robustness.keys(), closeness_robustness.values(), label="Closeness-based Deletions")
plt.plot(betweenness_robustness.keys(), betweenness_robustness.values(), label="Betweenness-based Deletions")

plt.xlabel("Number of Nodes Removed")
plt.ylabel("Size of Largest Connected Component")
plt.title("Network Robustness Comparison (Closeness vs. Betweenness)")
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'closeness_robustness' is not defined

<Figure size 1000x600 with 0 Axes>

### Question 7

Which measure is the fastest one to disconnect the network?
